In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d devesh1011/male-and-female-dataset

Dataset URL: https://www.kaggle.com/datasets/devesh1011/male-and-female-dataset
License(s): apache-2.0
100% 1.63G/1.63G [00:26<00:00, 15.6MB/s]
100% 1.63G/1.63G [00:26<00:00, 66.6MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/male-and-female-dataset.zip')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, AveragePooling2D, BatchNormalization
from keras.callbacks import EarlyStopping

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    directory="/content/Male and Female face dataset/train",
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256)
)

Found 4753 files belonging to 2 classes.


In [ ]:
test_ds = keras.utils.image_dataset_from_directory(
    directory="/content/Male and Female face dataset/test",
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256)
)

Found 665 files belonging to 2 classes.


In [ ]:
def process(image, label):
  image = tf.cast(image/255, tf.float32)
  return image, label

train_ds = train_ds.map(process)
test_ds = test_ds.map(process)

In [ ]:
model = Sequential([
    # First Convolutional Layer
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(256, 256, 3), padding='valid'),
    MaxPooling2D(pool_size=(2, 2), strides=2, padding="valid"),

    # Second Convolutional Layer
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='valid'),
    MaxPooling2D(pool_size=(2, 2), strides=2, padding="valid"),

    # Third Layer (Fully Connected Layer)
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid'),
    MaxPooling2D(pool_size=(2, 2), strides=2, padding="valid"),

    # Flatten before fully connected layers
    Flatten(),

    # Fully Connected Layer
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.25),

    Dense(64, activation='relu'),

    # Output Layer for 10 classes (e.g., digit classification)
    Dense(1, activation='sigmoid')
])

model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_21 (Conv2D)                   │ (None, 254, 254, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_22 (Conv2D)                   │ (None, 125, 125, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_23 (Conv2D)                   │ (None, 60, 60, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_7 (Flatten)                  │ (None, 115200)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 128)                 │      14,745,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,847,809 (56.64 MB)

 Trainable params: 14,847,553 (56.64 MB)

 Non-trainable params: 256 (1.00 KB)

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds, epochs=5, validation_data=test_ds, callbacks=[early_stopping])

Epoch 1/5
149/149 ━━━━━━━━━━━━━━━━━━━━ 98s 582ms/step - accuracy: 0.6903 - loss: 0.5843 - val_accuracy: 0.7098 - val_loss: 0.7896
Epoch 2/5
149/149 ━━━━━━━━━━━━━━━━━━━━ 86s 578ms/step - accuracy: 0.8055 - loss: 0.4084 - val_accuracy: 0.8301 - val_loss: 0.3843
Epoch 3/5
149/149 ━━━━━━━━━━━━━━━━━━━━ 85s 574ms/step - accuracy: 0.9060 - loss: 0.2416 - val_accuracy: 0.8677 - val_loss: 0.3077
Epoch 4/5
149/149 ━━━━━━━━━━━━━━━━━━━━ 86s 578ms/step - accuracy: 0.9633 - loss: 0.1084 - val_accuracy: 0.8045 - val_loss: 0.5849
Epoch 5/5
149/149 ━━━━━━━━━━━━━━━━━━━━ 86s 578ms/step - accuracy: 0.9841 - loss: 0.0607 - val_accuracy: 0.7609 - val_loss: 0.7948
